# 5-bus Market simulation with [PowerSimulations.jl](https://github.com/NREL-SIIP/PowerSimulations.jl)

**Originally Contributed by**: Clayton Barrows

## Introduction

PowerSimulations.jl supports simulations that consist of sequential optimization problems
where results from previous problems inform subsequent problems in a variety of ways. This
example demonstrates some of these capabilities to represent electricity market clearing.

## Dependencies and Data
First, let's create `System`s to represent the Day-Ahead and Real-Time market clearing
process with hourly, and 5-minute time series data, respectively.

### Modeling Packages

In [1]:
using PowerSystems
using PowerSimulations
const PSI = PowerSimulations
IS = PowerSystems.IS

InfrastructureSystems

### Data management packages

In [2]:
using Dates
using DataFrames

### Optimization packages

In [3]:
using Cbc # mip solver
solver = optimizer_with_attributes(Cbc.Optimizer, "logLevel" => 1, "ratioGap" => 0.5)
using Ipopt # solver that supports duals
ipopt_solver = optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)

MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute,Any}[MathOptInterface.RawParameter("print_level") => 0])

### 5-bus Data
The five bus system data here includes hourly day-ahead data, 5-minute real-time market
data, and 6-second actual data. We'll only use the hourly and 5-minute data for the
example simulations below, but the 6-second data is included for future development.

In [4]:
base_dir = PowerSystems.download(PowerSystems.TestData; branch = "master");
pm_data = PowerSystems.PowerModelsData(joinpath(base_dir, "matpower", "case5_re_uc.m"))

FORECASTS_DIR = joinpath(base_dir, "forecasts", "5bus_ts", "7day")

tsp_da = IS.read_time_series_file_metadata(
    joinpath(FORECASTS_DIR, "timeseries_pointers_da_7day.json"),
)
tsp_rt = IS.read_time_series_file_metadata(
    joinpath(FORECASTS_DIR, "timeseries_pointers_rt_7day.json"),
)
tsp_agc = IS.read_time_series_file_metadata(
    joinpath(FORECASTS_DIR, "timeseries_pointers_agc_7day.json"),
)

sys_DA = System(pm_data)
reserves = [
    VariableReserve{ReserveUp}("REG1", true, 5.0, 0.1),
    VariableReserve{ReserveUp}("REG2", true, 5.0, 0.06),
    VariableReserve{ReserveUp}("REG3", true, 5.0, 0.03),
    VariableReserve{ReserveUp}("REG4", true, 5.0, 0.02),
]
contributing_devices = get_components(Generator, sys_DA)
for r in reserves
    add_service!(sys_DA, r, contributing_devices)
end

add_time_series!(sys_DA, tsp_da)
transform_single_time_series!(sys_DA, 48, Hour(24))

sys_RT = System(pm_data)
add_time_series!(sys_RT, tsp_rt)
transform_single_time_series!(sys_RT, 12, Hour(1))

sys_AGC = System(pm_data)
add_time_series!(sys_AGC, tsp_agc)

┌ Info: extending matpower format with data: areas 1x3
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/pm_io/matpower.jl:362
┌ Info: extending matpower format with data: gen_name 7x4
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/pm_io/matpower.jl:362
┌ Info: extending matpower format by appending matrix "gen_name" in to "gen"
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/pm_io/matpower.jl:703
┌ Info: reversing the orientation of branch 6 (4, 3) to be consistent with other parallel branches
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/pm_io/data.jl:1222
┌ Info: the voltage setpoint on generator 4 does not match the value at bus 4
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/r86iN/src/parsers/pm_io/data.jl:1779
┌ Info: the voltage setpoint on generator 1 does not match the value at bus 1
└ @ PowerSystems /Users/cbarrows/.julia/packages/Po

## `OperationsProblemTemplate`s

In [5]:
template_uc = template_unit_commitment()
devices = Dict(
    :Generators => DeviceModel(ThermalStandard, ThermalDispatch),
    :Ren => DeviceModel(RenewableDispatch, RenewableFullDispatch),
    :Loads => DeviceModel(PowerLoad, StaticPowerLoad),
    :HydroROR => DeviceModel(HydroDispatch, HydroDispatchRunOfRiver),
    :RenFx => DeviceModel(RenewableFix, FixedOutput),
)
template_ed = template_economic_dispatch(devices = devices)


Operations Problem Specification

  transmission:  PowerSimulations.CopperPlatePowerModel
  devices: 
      HydroROR:
        device_type = PowerSystems.HydroDispatch
        formulation = PowerSimulations.HydroDispatchRunOfRiver
      Generators:
        device_type = PowerSystems.ThermalStandard
        formulation = PowerSimulations.ThermalDispatch
      Ren:
        device_type = PowerSystems.RenewableDispatch
        formulation = PowerSimulations.RenewableFullDispatch
      Loads:
        device_type = PowerSystems.PowerLoad
        formulation = PowerSimulations.StaticPowerLoad
      RenFx:
        device_type = PowerSystems.RenewableFix
        formulation = PowerSimulations.FixedOutput
  branches: 
      T:
        device_type = PowerSystems.Transformer2W
        formulation = PowerSimulations.StaticTransformer
      TT:
        device_type = PowerSystems.TapTransformer
        formulation = PowerSimulations.StaticTransformer
      L:
        device_type = PowerSystems.Line
 

### Define the Simulation Sequence

In [6]:
stages_definition = Dict(
    "UC" => Stage(
        GenericOpProblem,
        template_uc,
        sys_DA,
        solver,
        balance_slack_variables = true,
    ),
    "ED" => Stage(
        GenericOpProblem,
        template_ed,
        sys_RT,
        ipopt_solver,
        constraint_duals = [:CopperPlateBalance],
    ),
)

feedforward_chronologies = Dict(("UC" => "ED") => Synchronize(periods = 24))

feedforward = Dict(
    ("ED", :devices, :Generators) => SemiContinuousFF(
        binary_source_stage = PSI.ON,
        affected_variables = [PSI.ACTIVE_POWER],
    ),
)

cache = Dict("UC" => [TimeStatusChange(ThermalStandard, PSI.ON)])

order = Dict(1 => "UC", 2 => "ED")
horizons = Dict("UC" => 48, "ED" => 12)
intervals = Dict("UC" => (Hour(24), Consecutive()), "ED" => (Hour(1), Consecutive()))

DA_RT_sequence = SimulationSequence(
    step_resolution = Hour(24),
    order = order,
    horizons = horizons,
    intervals = intervals,
    ini_cond_chronology = InterStageChronology(),
    feedforward_chronologies = feedforward_chronologies,
    feedforward = feedforward,
)

Feed Forward Chronology
-----------------------

ED: PowerSimulations.SemiContinuousFF -> Generators

                     UC--┐ from : On
                         |
┌----┬----┬----┬----┬----┼----┬----┬----┬----┬----┬----┐
|    |    |    |    |    |    |    |    |    |    |    |
|    |    |    |    |    |    |    |    |    |    |    |
└─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED └─ED ... (x24) to : ["P"]

Initial Condition Chronology
----------------------------

1
|
|
2 --> 2 ... (x24)   


## `Simulation`

In [7]:
file_path = mkpath(joinpath(".","5-bus-simulation"))
sim = Simulation(
    name = "5bus-test",
    steps = 1,
    stages = stages_definition,
    stages_sequence = DA_RT_sequence,
    simulation_folder = file_path,
)

Simulation()


### Build simulation

In [8]:
build!(sim)

PowerSimulations.BuildStatusModule.BuildStatus.BUILT = 0

### Execute simulation
```julia

In [9]:
execute!(sim)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Nov  9 2020 

command line - Cbc_C_Interface -ratioGap 0.5 -logLevel 1 -solve -quit (default strategy 1)
ratioGap was changed from 0 to 0.5
Continuous objective value is 3.29855e+07 - 0.01 seconds
Cgl0004I processed model has 1208 rows, 2288 columns (710 integer (710 of which binary)) and 5496 elements
Cbc0045I MIPStart provided solution with cost 1.96663e+08
Cbc0012I Integer solution of 72203524 found by Reduced search after 0 iterations and 0 nodes (0.03 seconds)
Cbc0012I Integer solution of 37453229 found by DiveCoefficient after 0 iterations and 0 nodes (0.07 seconds)
Cbc0011I Exiting as integer gap of 4467772.5 less than 1e-10 or 50%
Cbc0001I Search completed - best objective 37453229.45153248, took 0 iterations and 0 nodes (0.07 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from 3.29855e+07 to 3.29855e+07
Probing was tried 0 times and created 0 cuts of which 0 w

PowerSimulations.RunStatusModule.RunStatus.SUCCESSFUL = 0

```

In [10]:
# Results

First we can load the result metadata
```julia
results = SimulationResults(sim);
uc_results = get_stage_results(results, "UC")
ed_results = get_stage_results(results, "ED");
```

Then we can read and examine the results of interest
```julia
prices = read_dual(ed_results, :CopperPlateBalance)
```

or if we want to look at the realized values
```julia
read_realized_duals(ed_results)[:CopperPlateBalance]
```

*note that in this simulation the prices are all equal to the balance slack
penalty value of $100000/MWh because there is unserved energy in the result*

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*